# Preparación de Datos

En esta sección se muestra el proceso de adquisición de los datos (importados desde un archivo excel), la creación de las variables objetivo que se necesitan para generar los modelos y la preparación de los datos para generar el ABT necesario para la construcción de los modelos predictivos.

### Librerías utilizadas

In [1]:
import pandas as pd
import numpy as np

# Para poder visualizar todas las columnas:
pd.set_option('display.max_columns',100)

# Para ver unicamente 3 decimales:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Acceso a los datos

Se importan los datos desde un archivo csv y se visializan los primeros registros para corroborar que se importaron correctamente:

In [2]:
data_inicial = pd.read_csv('C:/Users/gusta/OneDrive/Documentos/Tesis/Datos/Datos.csv') 
print(data_inicial.head())

   ID_EVENTO SEXO  EDAD  SMet    PAD     PAS  GLU  HDL  TRI     CC   PESO  \
0     623823    F    20     0 80.000 130.000   88   51   64 95.000 75.300   
1     623827    M    19     1 80.000 140.000   89   47  207 94.000 77.800   
2     623848    F    22     0 80.000 110.000   84   68   90 67.000 50.500   
3     623899    F    20     0 70.000 110.000   80   71   87 74.000 59.000   
4     623908    M    19     0 74.000 124.000  113   44  101 95.000 91.500   

    TALLA    IMC  C_TOTAL  HOMA    INS  CC_AGUA  CC_GRAS  CC_HUES  CC_MUSC  \
0 156.000 30.942      144 1.304  6.000   30.600   32.400    2.740    8.200   
1 166.500 28.064      161 3.428 15.600   39.700   26.100    3.550   10.700   
2 158.000 20.229      201 0.830  4.000   28.100   13.600    2.810    7.600   
3 157.000 23.936      183 0.751  3.800   32.000   11.800    2.810    8.300   
4 174.000 30.222      152 3.683 13.200   50.700   23.400    3.890   14.000   

   P_AGUA  P_GRAS  P_HUES  P_MUSC     VFA  
0  41.385  43.819   3.70

### Selección de variables 

Se seleccionan unicamente las medidas (variables) que no provienen de la química sanguínea, excepto las variables TRI y HDL, dado que la premisa es construir los modelos predictivos usando los valores de las medidas no invasivas.

In [3]:
data = data_inicial[[x for x in data_inicial.columns if x not in ['C_TOTAL','HOMA','INS']]]

### Perfilamiento de los datos

La cantidad de pacientes que presentan síndrome metabólico (SMet=1) vs aquellos que no lo tienen (SMet=0):

In [4]:
conteos = data['SMet'].value_counts()
porcentaje = data['SMet'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos, 'Porcentaje': porcentaje})

,Conteo,Porcentaje
0,1550,87.82%
1,215,12.18%


Se calculan los principales estadísticos a cada una de las variables analizadas a través de un método incluido en la librería de Pandas:

In [5]:
data.describe()

,ID_EVENTO,EDAD,SMet,PAD,PAS,GLU,HDL,TRI,CC,PESO,TALLA,IMC,CC_AGUA,CC_GRAS,CC_HUES,CC_MUSC,P_AGUA,P_GRAS,P_HUES,P_MUSC,VFA
count,1765.000,1765.000,1765.000,1758.000,1758.000,1765.000,1765.000,1765.000,1758.000,1753.000,1759.000,1751.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000,1765.000
mean,1902077.852,18.999,0.122,71.776,106.935,90.569,48.965,106.573,81.359,63.417,162.374,23.974,32.203,18.755,3.035,8.712,46.856,26.523,4.422,12.663,60.007
std,600394.926,1.438,0.327,8.729,11.740,8.177,10.310,56.874,11.191,13.609,8.261,4.343,7.021,8.684,0.686,1.952,16.630,12.604,1.626,4.507,34.360
min,623823.000,18.000,0.000,40.000,64.000,60.000,17.000,22.000,42.500,38.800,137.000,14.226,17.300,1.500,-7.060,4.800,0.338,0.030,-11.623,0.092,5.000
25%,1793376.000,18.000,0.000,66.000,100.000,86.000,42.000,71.000,73.500,53.400,156.000,20.838,26.900,12.600,2.590,7.200,44.778,19.317,4.312,12.100,38.200
50%,2095692.000,18.000,0.000,70.000,110.000,90.000,48.000,91.000,80.000,61.000,162.000,23.203,30.300,17.400,2.910,8.200,50.086,28.767,4.847,13.532,56.800
75%,2120448.000,19.000,0.000,80.000,112.000,95.000,55.000,126.000,87.500,71.000,168.000,26.505,36.900,23.800,3.420,10.000,56.049,35.793,5.311,15.164,78.300
max,5642826.000,24.000,1.000,120.000,160.000,220.000,113.000,698.000,126.000,129.800,188.000,44.139,67.800,65.300,6.220,19.100,86.763,53.038,7.619,21.891,479.600


Del procedimiento anterior se identifica que hay registros en los que se tienen missing values en alguna de las variables. A continuación, se filtraran esos registros para que sean analizados:

In [6]:
nulos = data.isnull().values.any(axis=1)
data_nulos = data[nulos]
data_nulos

,ID_EVENTO,SEXO,EDAD,SMet,PAD,PAS,GLU,HDL,TRI,CC,PESO,TALLA,IMC,CC_AGUA,CC_GRAS,CC_HUES,CC_MUSC,P_AGUA,P_GRAS,P_HUES,P_MUSC,VFA
169,911364,F,18,0,nan,nan,87,43,175,nan,80.700,nan,nan,31.300,37.900,2.910,8.600,38.781,46.958,3.606,10.655,71.600
216,1159105,F,19,0,nan,nan,91,47,188,nan,51.900,nan,nan,24.800,18.000,2.350,6.800,47.738,34.649,4.524,13.090,62.300
509,1793514,F,23,0,nan,nan,93,56,82,nan,nan,nan,nan,29.000,34.100,2.990,7.700,39.301,46.212,4.052,10.435,96.400
631,1794686,M,18,0,nan,nan,99,65,80,81.000,nan,171.000,nan,39.800,12.600,3.730,11.000,59.288,18.770,5.556,16.386,36.200
822,1794992,F,19,0,62.000,88.000,99,68,123,79.000,nan,164.000,nan,33.500,26.500,3.290,9.100,46.277,36.607,4.545,12.571,71.600
900,2119942,F,18,0,60.000,100.000,81,51,63,71.000,nan,164.000,nan,28.400,11.500,2.700,7.600,56.574,22.908,5.378,15.139,39.000
940,2119988,F,18,0,60.000,104.000,95,38,82,80.000,nan,153.000,nan,27.400,26.400,2.610,7.400,42.940,41.373,4.090,11.597,78.300
943,2119992,F,18,0,72.000,110.000,92,69,69,74.000,nan,164.000,nan,32.300,19.800,3.200,8.900,50.312,30.841,4.984,13.863,82.700
1075,2120144,M,18,0,78.000,114.000,97,51,89,74.000,nan,177.000,nan,41.800,7.200,3.850,11.500,64.957,11.189,5.983,17.871,5.800
1520,2120717,F,18,0,nan,nan,82,43,58,nan,nan,nan,nan,29.800,19.000,2.960,7.800,50.034,31.901,4.970,13.096,53.900


Se identifica que todos los pacientes en los que se tienen missing values no tienen síndrome metabólico y que este subconjunto de pacientes representa el 1% de la población total, por lo que eliminar aquellos registros con missing values no representará una pérdida significativa de información y de esta forma no se provoca un sesgo en el análisis al utilizar algún método de imputación de valores:

In [7]:
data = data.dropna()
conteos = data['SMet'].value_counts()
porcentaje = data['SMet'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos, 'Porcentaje': porcentaje})

,Conteo,Porcentaje
0,1535,87.71%
1,215,12.29%


Se calculan nuevamente los principales estadísticos a cada una de las variables consideradas:

In [8]:
data.describe()

,ID_EVENTO,EDAD,SMet,PAD,PAS,GLU,HDL,TRI,CC,PESO,TALLA,IMC,CC_AGUA,CC_GRAS,CC_HUES,CC_MUSC,P_AGUA,P_GRAS,P_HUES,P_MUSC,VFA
count,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000,1750.000
mean,1902005.260,19.002,0.123,71.820,106.962,90.576,48.946,106.636,81.379,63.416,162.361,23.974,32.211,18.734,3.035,8.714,46.835,26.471,4.419,12.658,60.013
std,601993.568,1.438,0.328,8.706,11.742,8.184,10.301,56.976,11.208,13.612,8.266,4.344,7.038,8.684,0.688,1.956,16.687,12.615,1.632,4.523,34.437
min,623823.000,18.000,0.000,40.000,64.000,60.000,17.000,22.000,42.500,38.800,137.000,14.226,17.300,1.500,-7.060,4.800,0.338,0.030,-11.623,0.092,5.000
25%,1793371.000,18.000,0.000,66.000,100.000,86.000,42.000,71.000,73.500,53.400,156.000,20.836,26.900,12.600,2.590,7.200,44.761,19.209,4.311,12.098,38.125
50%,2095687.500,18.000,0.000,70.000,110.000,90.000,48.000,91.000,80.000,61.000,162.000,23.202,30.300,17.350,2.910,8.200,50.101,28.646,4.850,13.534,56.800
75%,2120445.750,19.000,0.000,80.000,112.000,95.000,55.000,126.000,87.500,71.000,168.000,26.505,36.900,23.700,3.420,10.000,56.068,35.748,5.311,15.165,78.300
max,5642826.000,24.000,1.000,120.000,160.000,220.000,113.000,698.000,126.000,129.800,188.000,44.139,67.800,65.300,6.220,19.100,86.763,53.038,7.619,21.891,479.600


### Construcción de variables objetivo

Uno de los criterios para diagnosticar síndrome metabólico es que los triglicéridos estén por arriba de 150 mg/dL:

In [9]:
data['TARGET_TRI'] = np.where(data['TRI']>150,1,0)
conteos_tri = data['TARGET_TRI'].value_counts()
porcentaje_tri = data['TARGET_TRI'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos_tri, 'Porcentaje': porcentaje_tri})

,Conteo,Porcentaje
0,1472,84.11%
1,278,15.89%


Otro de los criterios para diagnosticar síndrome metabólico es que el colesterol HDL esté por debajo de 40 mg/dL en hombres y de 50 mg/dL en mujeres. Como no se cuenta con suficientes datos para hacer un modelo para cada segmento, se tomará como punto de corte para ambos sexos que el colesterol HDL esté por debajo de 50 mg/dL.

In [10]:
data['TARGET_HDL'] = np.where(data['HDL']<40 ,1,0)
conteos_hdl = data['TARGET_HDL'].value_counts()
porcentaje_hdl = data['TARGET_HDL'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'Conteo': conteos_hdl, 'Porcentaje': porcentaje_hdl})

,Conteo,Porcentaje
0,1443,82.46%
1,307,17.54%
